In [1]:
import requests

In [3]:
URL = 'https://accounts.spotify.com/api/token'

In [4]:
requests.get(URL)

<Response [405]>

In [ ]:
def getAccesToken(client_id, client_secret):
    message = f'{client_id:client_secret}'
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii')
    
    aut_header['Authorization'] = 'Basic ' + base64_message
    aut_data = 'client_credentials'
    
    res = request.post(URL,  headers = aut_header, data = aut_data)
    
    response_obj = res.json()
    
    acces_token = response_obj['acces token']
    
    return acces_token

def getPlaylistTracks():